In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Question 5: Sales ROI: What is the average profit expectation from upgrades and sales after mortgage payments? (10 points)

In [3]:
# Load the CSV files into variables
file_paths = {
    "growth": "./Metro_zhvf_growth_uc_sfrcondo_tier_0.33_0.67_month.csv",
    "value": "./Metro_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_month.csv",
    "median_price": "./Metro_median_sale_price_uc_sfrcondo_month.csv",
    "list_price": "./Metro_mlp_uc_sfrcondo_month.csv",
    "sold_above_list": "./Metro_pct_sold_above_list_uc_sfrcondo_month.csv",
    "sold_below_list": "./Metro_pct_sold_below_list_uc_sfrcondo_month.csv"
}

# Loaded CSV variables into pandas DataFrames
dataframes = {name: pd.read_csv(path) for name, path in file_paths.items()}

# Display only the first 5 rows of each DataFrame using head()
dataframes_heads = {name: df.head() for name, df in dataframes.items()}

# Output the first 5 rows for each dataset
for name, head in dataframes_heads.items():
    print(f"First 5 rows of {name} dataset:")
    print(head)
    print("\n")

First 5 rows of growth dataset:
   RegionID  SizeRank       RegionName RegionType StateName    BaseDate  \
0    394913         1     New York, NY        msa        NY  10/31/2024   
1    753899         2  Los Angeles, CA        msa        CA  10/31/2024   
2    394463         3      Chicago, IL        msa        IL  10/31/2024   
3    394514         4       Dallas, TX        msa        TX  10/31/2024   
4    394692         5      Houston, TX        msa        TX  10/31/2024   

   11/30/2024  1/31/2025  10/31/2025  
0        -0.5       -0.8         1.1  
1        -0.7       -0.8         2.2  
2        -1.0       -2.2         0.7  
3        -1.1       -2.1         1.3  
4        -0.9       -1.7         0.8  


First 5 rows of value dataset:
   RegionID  SizeRank       RegionName RegionType StateName    1/31/2023  \
0    394913         1     New York, NY        msa        NY  618315.3511   
1    753899         2  Los Angeles, CA        msa        CA  903986.6773   
2    394463         3 